In [8]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
from scipy.integrate import odeint
import scipy.integrate as spi
import scipy.stats as st
from array import *
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt
import scipy.io as io
import math
import random


In [9]:
# -- Functions for loading paramters from csv files
def GetPopParams(Popparameters, dose):
    ''' 
    input:
        - data frame with parameter values as columns, individuals as rows

    return:
        - dictionary with parameter names as key and parameter values as values
    '''
    cols = list(Popparameters.parameter)
   # print(cols)
    pid_params = Popparameters
  #  print(pid_params.value)
    values = []
    keys = []
    for i, c in enumerate(cols):
        if 'pop' in c:
            k = c.split('_')[0]
            keys.append(k)
         #   print(keys)
            v = pid_params.value[i]
            values.append(v)
         #   print(values)
            
        for i,(v,k) in enumerate(zip(values, keys)):
            if 'log10' in k:
                values[i] = 10**v
                keys[i] = k.split('0')[1]
    if 'alpha' in keys:
        i = keys.index('alpha')
        j = keys.index('kPL')
        values[j] = values[j]*dose**values[i]

    params = dict(zip(keys, values))
    return params

def GetPopParamsSE(Popparameters, dose):
    ''' 
    input:
        - data frame with parameter values as columns, individuals as rows

    return:
        - dictionary with parameter names as key and parameter values as values
    '''
    cols = list(Popparameters.parameter)
   # print(cols)
    pid_params = Popparameters
  #  print(pid_params.value)
    SE = []
    keys = []
    for i, c in enumerate(cols):
        if 'pop' in c:
            k = c.split('_')[0]
            keys.append(k)
         #   print(keys)
            se = pid_params.se_sa[i]
            SE.append(se)
         #   print(values)
            

    paramsSE = dict(zip(keys, SE))
    return paramsSE


def GetPopParamsOmega(Popparameters, dose):
    ''' 
    input:
        - data frame with parameter values as columns, individuals as rows

    return:
        - dictionary with parameter names as key and parameter values as values
    '''
    cols = list(Popparameters.parameter)
   # print(cols)
    pid_params = Popparameters
  #  print(pid_params.value)
    omega = []
    keys = []
    for i, c in enumerate(cols):
        if 'omega' in c:
            k = c.split('_')[1]
            keys.append(k)
         #   print(keys)
            om = pid_params.value[i]
            omega.append(om)
         #   print(values)
            

    paramsOmega = dict(zip(keys, omega))
    return paramsOmega

def GetVLIndParams(parameters_df, ID, style = 'mode'):
    ''' 
    input:
        - data frame with parameter values as columns, individuals as rows
        - ID identifying individual of interest
        - style = 'mode' or 'mean' or 'SAEM' depending on which parameter values are preferred
    return:
        - dictionary with parameter names as key and parameter values as values
    '''
    i = ID
    cols = list(parameters_df)
    pid_params = parameters_df[parameters_df.id == i]
    values = []
    keys = []
    for c in cols:
        if style in c:
            k = c.split('_')[0]
            keys.append(k)
            v = pid_params[c].values[0]
            values.append(v)
    for i,(v,k) in enumerate(zip(values, keys)):
        if 'log10' in k:
            values[i] = 10**v
            keys[i] = k.split('0')[1]

    params = dict(zip(keys, values))
    return params

def RandVLIndParams(parameters_df, ID):
    ''' 
    input:
        - data frame with parameter values as columns, individuals as rows
        - ID identifying individual of interest
        - style = 'mode' or 'mean' or 'SAEM' depending on which parameter values are preferred
    return:
        - dictionary with parameter names as key and parameter values as values
    '''
    i = ID
    cols = list(parameters_df)
    pid_params = parameters_df[parameters_df.id == i]
    mean = []
    STDEV = []
    values = []
    keys = []
    for c in cols:
        if 'mean' in c:
            k = c.split('_')[0]
            keys.append(k)
            mn = pid_params[c].values[0]
            mean.append(mn)
        if 'sd' in c:
            s = pid_params[c].values[0]
            STDEV.append(s)
    values=np.random.normal(mean,STDEV, len(STDEV))

    for i,(v,k) in enumerate(zip(values, keys)):
        if k=='h' and values[i]<0:
            values[i]= -values[i]
        if 'log10' in k:
            values[i] = 10**v
            keys[i] = k.split('0')[1]

    params = dict(zip(keys, values))
    return params

def RandVLParams(Popparameters, param_dist):
    
    cols = list(Popparameters.parameter)
    
    pid_params = Popparameters
    
    omega = []
    values = []
    keys = []
    for i, c in enumerate(cols):
        if 'pop' in c:
            k = c.split('_')[0]
            keys.append(k)
         #   print(keys)
            v = pid_params.value[i]
            values.append(v)
        if 'omega' in c:
            om = pid_params.value[i]
            omega.append(om)
         #   print(values)
    eta = np.random.normal(np.zeros(len(omega)), omega, len(omega))   
    
    for i,(v,k) in enumerate(zip(values, keys)):
        if param_dist[k]=='logNormal':  
            values[i] = v*np.exp(eta[i])
        elif 'log10' in k:
            values[i]=v+eta[i]
            values[i] = 10**values[i]
            keys[i] = k.split('0')[1]
        else:
            minn = param_dist[k+'min']
            maxx = param_dist[k+'max']
            v = np.log((v-minn)/(maxx-v)) + eta[i]
            values[i] = (maxx*np.exp(v)+minn)/(1+np.exp(v))

    params = dict(zip(keys, values))
    return params

In [10]:
def Cohort_Prep(IDs, Num):
    Cohort = random.choices(IDs, k=Num)
    return np.asarray(Cohort)


def PKPDParams(Popparameters, GetPopParams, GetParamsOmega, Emax, IC50, Hill, dose, PDOm):
    
    param_Order = ['ka', 'k12', 'k21', 'kcl', 'Vol', 'Emax', 'IC50', 'Hill']
    PKparams = GetPopParams(Popparameters, dose)
    PKparams_Val = PKparams['ka'], PKparams['kPL'], PKparams['kLP'], PKparams['kCl'],PKparams['Vol']
    PDparams = Emax, IC50, Hill
    PKPDparams = np.hstack((PKparams_Val, PDparams))
    
    PKOm = GetParamsOmega(Popparameters, dose)
    PKOm_Val = PKOm['ka'], PKOm['kPL'], PKOm['kLP'], PKOm['kCl'],PKOm['Vol']
    PKPDOm = np.hstack((PKOm_Val, PDOm))
    eta = np.random.normal(np.zeros(len(PKPDOm)), PKPDOm, len(PKPDOm))
    PKPDInd = PKPDparams*np.exp(eta)
    return dict(zip(param_Order, PKPDInd))

def ParamsPrep(Cohort, CohortStyle, GetVLIndParams, RandVLParams, parameters_df, Popparameters_df, fixed_params, param_order, param_dist,
               PKPDParams, Popparameters_PK, GetPopParams, GetPopParamsOmega, Emax, IC50, Hill,PDOm, dose):

    VLparams = []
    PKPD_params = []
    param_dict = []
    for ID in Cohort:
        if CohortStyle == 'direct':
            ind_params = GetVLIndParams(parameters_df, ID, style = 'mode')
        else:
            ind_params = RandVLParams(Popparameters_df, param_dist)
            
        _dict = {**fixed_params, **ind_params}
        param_dict.append(_dict)
        params = []
        for k in param_order:
            params.append(_dict[k])
        VLparams.append(params)
        pkpd =PKPDParams(Popparameters_PK, GetPopParams, GetPopParamsOmega, Emax, IC50, Hill, dose, PDOm)
        PKPD_params.append(pkpd)
    return VLparams, PKPD_params, param_dict

In [14]:
# Write function for model simulation
def VLModel(y, t, beta, phi, rho, k, delta,h, m, pi, c,tAI):
    '''Natural History model'''
    T,R,E,I,V = y
    
    if t > tAI:
        eps = m
    else:
        eps = 0
        
    ddt_T = -beta*T*V - phi*I*T + rho*R
    ddt_R = phi*I*T -rho*R
    ddt_E = beta*T*V - k*E
    ddt_I = k*E - delta*np.maximum(I,.000001)**h*I - eps*I
    if I>=1:
        ddt_V = pi*I-c*V
    else:
        ddt_V = -c*V
    return ddt_T, ddt_R, ddt_E, ddt_I, ddt_V


# Write function for model simulation
def VLPKPD(y, t, beta, phi, rho, k, delta,h, m, pi, c,tAI, ka, k12, kcl, k21,Vol, MolMass,E_max, IC50, Hill_Coeff):
    T,R,E,I,V,Ag, A1, A2 = y
    
    # PK model 
    dAg = -ka*Ag
    dA1 = ka*Ag+k21*A2-(kcl+k12)*A1
    dA2 = k12*A1-k21*A2   
    
    # PD model 
    conc = A1*10**6/Vol/MolMass # # convert to from nanogram/mL to micromolar
    epsN = np.divide(np.multiply(E_max,np.power(conc,Hill_Coeff)),(np.power(IC50, Hill_Coeff)+np.power(conc,Hill_Coeff)))
        
    # within-host treatment model
    if t > tAI:
        m_AI = m
    else:
        m_AI = 0
        
    ddt_T = -beta*T*V - phi*I*T + rho*R
    ddt_R = phi*I*T -rho*R
    ddt_E = beta*T*V - k*E
    ddt_I = k*E - delta*np.maximum(I,.000001)**h*I - m_AI*I
    if I>=1:
        ddt_V = (1-epsN)*pi*I-c*V
    else:
        ddt_V = -c*V

    return  ddt_T, ddt_R, ddt_E, ddt_I, ddt_V, dAg, dA1, dA2

def PK_Model(y,t, ka, k12, kcl, k21):
    Ag, A1, A2 = y
    
    dAg = -ka*Ag
    dA1 = ka*Ag+k21*A2-(kcl+k12)*A1
    dA2 = k12*A1-k21*A2
    #print(dAg)
    return dAg, dA1, dA2

def PD_Model(Amnt, MolMass, PKPDInd, PotRed):
    #PD parameters
    E_max = PKPDInd['Emax']
    Hill_Coeff = PKPDInd['Hill']
    IC50 = PotRed*PKPDInd['IC50']
    
    Vol = PKPDInd['Vol']
    Conc = Amnt*10**6/Vol/MolMass
    
    E = np.divide(np.multiply(E_max,np.power(Conc,Hill_Coeff)),(np.power(IC50, Hill_Coeff)+np.power(Conc,Hill_Coeff)))
    return E

# Write function for setting initial conditions for within-host model from the parameter dictionary

def SetInit(param_dict):
    T_0 = 10**7
    R_0 = 0;
    E_0 = 1;
    I_0 = 0
    V_0 = 100
    #param_dict['pi']*I_0/param_dict['c']
    return [T_0, R_0, E_0, I_0, V_0]

def SimulateTreatment(VLPKPD, init, tzero, dt, tend, params, PKPDInd, dose, dosetimes, PotRed, MolMass):
    #Nirmatrelvir PD parameters
    E_max = PKPDInd['Emax']
    Hill_Coeff = PKPDInd['Hill']
    IC50 = PotRed*PKPDInd['IC50']

    #Nirmatrelvir PK Parameters for each individual
    ka = PKPDInd['ka']
    k12 = PKPDInd['k12']
    k21 = PKPDInd['k21']
    kcl = PKPDInd['kcl']
    Vol = PKPDInd['Vol']
    params_local = params.copy()
    params_local.extend([ka, k12, kcl, k21,Vol, MolMass, E_max, IC50, Hill_Coeff])
    args = tuple(params_local)
  
    if tzero==dosetimes[0]:
        for j in range(0,len(dosetimes)-1):
            init[5] = init[5] + dose 
            ttemp = np.arange(dosetimes[j],dosetimes[j+1],dt)
            ytemp = spi.odeint(VLPKPD, init, ttemp, args = args)
            init = ytemp[-1,:].T
            if j==0:
                y= ytemp.copy()
                t= ttemp.copy()
            else:
                y = np.concatenate((y,ytemp), axis=0)
                t = np.concatenate((t,ttemp))
    else:       
        
        # simulate infection from tzero up to 1st dose
        t = np.arange(tzero, dosetimes[0],dt)
        # simulate using scipy integrate
        y = spi.odeint(VLPKPD, init, t, args = args)
   
        ytemp = y.copy()

        #loop through dose values
        for j in range(0,len(dosetimes)-1):
            init = ytemp[-1,:].T
            init[5] = init[5] + dose 
            ttemp = np.arange(dosetimes[j],dosetimes[j+1],dt)
        
            ytemp = spi.odeint(VLPKPD, init, ttemp, args = args)
            t=np.concatenate((t,ttemp))
            y=np.concatenate((y,ytemp),axis=0)        
    #simulate until tend
    init = ytemp[-1,:].T
    init[5] = init[5] + dose 
    ttemp = np.arange(dosetimes[-1],tend,dt)
    ytemp = spi.odeint(VLPKPD, init, ttemp, args = args)
    t=np.concatenate((t,ttemp))
    y=np.concatenate((y,ytemp),axis=0)  
    return t, y

def SimulateTreatment1(VLPKPD, init, tzero, dt, tend, params, PKPDInd, dose, dosetimes, PotRed, MolMass):
    #Nirmatrelvir PD parameters
    E_max = PKPDInd['Emax']
    Hill_Coeff = PKPDInd['Hill']
    IC50 = PotRed*PKPDInd['IC50']

    #Nirmatrelvir PK Parameters for each individual
    ka = PKPDInd['ka']
    k12 = PKPDInd['k12']
    k21 = PKPDInd['k21']
    kcl = PKPDInd['kcl']
    Vol = PKPDInd['Vol']
    params_local = params.copy()
    params_local.extend([ka, k12, kcl, k21,Vol, MolMass, E_max, IC50, Hill_Coeff])
    args = tuple(params_local)
    
    # simulate infection from tzero up to 1st dose
    t = np.arange(tzero, dosetimes[0],dt)

    # simulate using scipy integrate
    y = spi.odeint(VLPKPD, init, t, args = args)
    ytemp = y.copy()

    #loop through dose values
    for j in range(0,len(dosetimes)-1):
        init = ytemp[-1,:].T
        init[5] = init[5] + dose 
        ttemp = np.arange(dosetimes[j],dosetimes[j+1],dt)
        
        ytemp = spi.odeint(VLPKPD, init, ttemp, args = args)
        t = np.concatenate((t,ttemp))
        y = np.concatenate((y,ytemp), axis = 0)         
    #simulate until tend
    init = ytemp[-1,:].T
    init[5] = init[5] + dose 
    ttemp = np.arange(dosetimes[-1],tend,dt)
    ytemp = spi.odeint(VLPKPD, init, ttemp, args = args)
    t = np.concatenate((t,ttemp))
    y = np.concatenate((y,ytemp), axis = 0) 
    return t, y

def SimulateTreatmentPKPD(PK_Model, init, tzero, dt, tend, PKPDInd, dose, dosetimes, PotRed, MolMass):
    #Nirmatrelvir PD parameters
    E_max = PKPDInd['Emax']
    Hill_Coeff = PKPDInd['Hill']
    IC50 = PotRed*PKPDInd['IC50']

    #Nirmatrelvir PK Parameters for each individual
    ka = PKPDInd['ka']
    k12 = PKPDInd['k12']
    k21 = PKPDInd['k21']
    kcl = PKPDInd['kcl']
    Vol = PKPDInd['Vol']
    PKparams = [ka, k12, kcl, k21]
    args = tuple(PKparams)
    
    #loop through dose values
    for j in range(0,len(dosetimes)-1):
        init[0] = init[0] + dose 
        ttemp = np.arange(dosetimes[j],dosetimes[j+1],dt)
        ytemp = spi.odeint(PK_Model, init, ttemp, args = args)
        if j==0:
            y=ytemp
            t=ttemp
        else:
            t = np.concatenate((t,ttemp))
            y = np.concatenate((y,ytemp), axis = 0)     
        init = ytemp[-1,:].T
    #simulate until tend
    init = ytemp[-1,:].T
    init[0] = init[0] + dose 
    ttemp = np.arange(dosetimes[-1],tend,dt)
    ytemp = spi.odeint(PK_Model, init, ttemp, args = args)
    t = np.concatenate((t,ttemp))
    y = np.concatenate((y,ytemp), axis = 0) 
    conc = y[:,1]*10**6/Vol/MolMass # # convert to from nanogram/mL to micromolar
    epsN = np.divide(np.multiply(E_max,np.power(conc,Hill_Coeff)),(np.power(IC50, Hill_Coeff)+np.power(conc,Hill_Coeff)))
    return t, y, epsN

def SimulateTreatmentPK(PK_Model, init, tzero, dt, tend, PKPDInd, dose, dosetimes):

    #Nirmatrelvir PK Parameters for each individual
    ka = PKPDInd['ka']
    k12 = PKPDInd['k12']
    k21 = PKPDInd['k21']
    kcl = PKPDInd['kcl']
    Vol = PKPDInd['Vol']
    PKparams = [ka, k12, kcl, k21]
    args = tuple(PKparams)
    
    #loop through dose values
    for j in range(0,len(dosetimes)-1):
        init[0] = init[0] + dose 
        ttemp = np.arange(dosetimes[j],dosetimes[j+1],dt)
        ytemp = spi.odeint(PK_Model, init, ttemp, args = args)
        if j==0:
            y=ytemp
            t=ttemp
        else:
            t = np.concatenate((t,ttemp))
            y = np.concatenate((y,ytemp), axis = 0)     
        init = ytemp[-1,:].T
    #simulate until tend
    init = ytemp[-1,:].T
    init[0] = init[0] + dose 
    ttemp = np.arange(dosetimes[-1],tend,dt)
    ytemp = spi.odeint(PK_Model, init, ttemp, args = args)
    t = np.concatenate((t,ttemp))
    y = np.concatenate((y,ytemp), axis = 0)
    return t, y

def TreatmentImpact(V,t,TrtmntStrtDay, dosetimes):
    t_on = np.argwhere(t >= TrtmntStrtDay)[0][0]#index of treatment start
    t_off = np.argwhere(t >= dosetimes[-1])[0][0]#index of treatment end
    t_10 = np.argwhere(t>=TrtmntStrtDay+10)[0][0]#index of time 10 days after treatment start day
    t_3 = np.argwhere(t>=TrtmntStrtDay+3)[0][0]#index of time 3 days after treatment start day
    t_2 = np.argwhere(t>=TrtmntStrtDay+2)[0][0]#index of time 2 days after treatment start day
    t_14 = np.argwhere(t>=TrtmntStrtDay+14)[0][0]#index of time 3 days after treatment start day
    t_final = np.argwhere(t == t[-1])[0][0]
    LVt = np.log10(np.maximum(V,10**(2.65)))
    
    # VL at treatment onset - VL w/ treatment until the end of simulation (t_end)
    drop_VL = LVt[t_on] - LVt[t_on:t_final]
    
    drop_VL_two = LVt[t_2]-LVt[t_on]
    drop_VL_three = LVt[t_3]-LVt[t_on]
    drop_VL_five = LVt[t_off]-LVt[t_on]
    drop_VL_ten = LVt[t_10]-LVt[t_on]
    drop_VL_fourteen = LVt[t_14]-LVt[t_on]
    #print(LVt)
    return drop_VL_two, drop_VL_three, drop_VL_five, drop_VL_ten, drop_VL_fourteen

def CI_Calc(Mean_drop_VL_five_Cntrl, Mean_drop_VL_five, Std_drop_VL_five_Cntrl, Std_drop_VL_five, IDs, n):
    Mean_VL_drop_Diff = Mean_drop_VL_five_Cntrl-Mean_drop_VL_five
    Var_drop_VL_five = [i**2 for i in Std_drop_VL_five]
    Var_drop_VL_five_Cntrl = Std_drop_VL_five_Cntrl**2
    Std_VL_drop_Diff = np.sqrt(Var_drop_VL_five+Var_drop_VL_five_Cntrl)
    df = len(IDs)-n-1
    return [list(st.norm.interval(alpha=0.95, loc=mean, scale=std/np.sqrt(df))) for mean, std in zip(Mean_VL_drop_Diff,Std_VL_drop_Diff)]

def Trial_Simulation(GetVLIndParams, RandVLParams, parameters_df, Popparameters_df, ID, TrtmntStrtDay, PotRed, 
                     fixed_params, param_order, param_dist, PKPDParams, Popparameters_PK, 
                     GetPopParams, GetPopParamsOmega, Emax, IC50, Hill, MolMass, 
                     dose, PDOm, SetInit, VLPKPD, CohortStyle, 
                     TreatmentLength = 5, TreatmentFrequency = 0.5):
        # import parameters
        if CohortStyle == 'direct':
            ind_params = GetVLIndParams(parameters_df, ID, style = 'mode')
        else:
            ind_params = RandVLParams(Popparameters_df, param_dist)
            
        param_dict = {**fixed_params, **ind_params}

        PKPDInd = PKPDParams(Popparameters_PK, GetPopParams, GetPopParamsOmega, Emax, IC50, Hill, dose, PDOm)
        params = []
        for k in param_order:
            params.append(param_dict[k])
        # extract initial time
        t0 = -param_dict['tzero']
        # extract onset of symptoms from data
       # t_symp = filtered.SympOnsetDelay[filtered.ID == ID]
  #      TrtmntStrtDay = random.choice(range(0,4)) #treatment starting within 3 days of synptom onset
        t_end = 20 #duration of simulation 20 days after symptom onset. 
        #TrtmntStrtDay+14 # the duration of simulation (14 days after the start of treatment)
        init = SetInit(param_dict)
        init.extend([0,0,0])
        
        dosetimes = TrtmntStrtDay + np.arange(0,TreatmentLength,TreatmentFrequency)
     #   print(dosetimes[0])
     #   if (t0 > t_symp.values[0]):
      #      n=n+1
      #      continue
        t,y = SimulateTreatment(VLPKPD, init, t0, 0.01, t_end, params, PKPDInd, dose, dosetimes, PotRed, MolMass)
        return t, y, dosetimes
    
def Trial_Simulation_paramInput(TrtmntStrtDay, params, 
                                PKPDInd, param_dict, PotRed, MolMass, dose, SetInit, VLPKPD,
                               TreatmentLength = 5, TreatmentFrequency = 0.5):
        # import parameters
        # extract initial time
        t0 = -param_dict['tzero']
        # extract onset of symptoms from data
       # t_symp = filtered.SympOnsetDelay[filtered.ID == ID]
  #      TrtmntStrtDay = random.choice(range(0,4)) #treatment starting within 3 days of synptom onset
        t_end = 30 #duration of simulation 30 days after symptom onset. 
        #TrtmntStrtDay+14 # the duration of simulation (14 days after the start of treatment)
        init = SetInit(param_dict)
        init.extend([0,0,0])
        
        dosetimes = TrtmntStrtDay + np.arange(0,TreatmentLength,TreatmentFrequency)
     #   print(dosetimes[0])
     #   if (t0 > t_symp.values[0]):
      #      n=n+1
      #      continue
        t,y = SimulateTreatment(VLPKPD, init, t0, 0.001, t_end, params, PKPDInd, dose, dosetimes, PotRed, MolMass)
        return t, y, dosetimes